In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import uuid
from sklearn import cross_validation
from datetime import date, timedelta
from sklearn.cross_validation import KFold, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy.stats import pearsonr
import numpy as np
from sklearn.cluster import AgglomerativeClustering

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print("Load the training, test and store data using pandas")
train = pd.read_csv("../input/train.csv")
test  = pd.read_csv("../input/test.csv")
store = pd.read_csv("../input/store.csv")
train.head()

Load the training, test and store data using pandas


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [3]:
def pearson_affinity(M):
    return 1 - np.array([[pearsonr(a,b)[0] for a in M] for b in M])

def factor(series):
    #input should be a pandas series object
    dic = {}
    for i,val in enumerate(series.value_counts().index):
        dic[val] = i
    return [ dic[val] for val in series.values ]  


def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w


def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe


def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe


In [4]:
def XGBoost(X_train,X_valid,params,verbose=False):
    dtrain = xgb.DMatrix(X_train[features], np.log(X_train["Sales"] + 1))
    dvalid = xgb.DMatrix(X_valid[features], np.log(X_valid["Sales"] + 1))
    num_trees = params['num_trees']
    
    watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
    #watchlist = [(dvalid, 'eval'),(dtrain, 'train')]

    gbm = xgb.train(params,
                    dtrain,
                    num_trees,
                    evals=watchlist,
                    early_stopping_rounds=10,
                    feval=rmspe_xg,
                    verbose_eval=verbose,
                    )
    
    train_probs = gbm.predict(xgb.DMatrix(X_train[features]),ntree_limit=gbm.best_iteration)

    train_error = rmspe(np.exp(train_probs) - 1, X_train['Sales'].values)
    
    valid_probs = gbm.predict(xgb.DMatrix(X_valid[features]),ntree_limit=gbm.best_iteration)
    indices = valid_probs < 0
    valid_probs[indices] = 0
    valid_error = rmspe(np.exp(valid_probs) - 1, X_valid['Sales'].values)
    return gbm, valid_error, train_error



In [5]:
# Gather some features
def build_features(features, data, dates):
    # remove NaNs
    data.fillna(0, inplace=True)
    data.loc[data.Open.isnull(), 'Open'] = 1
        
    # Use some properties directly
    features.extend(['Store', 'CustomerCluster', 'SalesCluster','CompetitionOpenSinceMonth',
                     'CompetitionOpenSinceYear', 'Promo', 'Promo2SinceWeek', 'Promo2SinceYear'])
    
    #log of CompetitionDistance
    features.append('logDist')
    data['logDist'] = np.log(1+data.CompetitionDistance)

    # add some more with a bit of preprocessing
    features.append('SchoolHoliday')
    data['SchoolHoliday'] = data['SchoolHoliday'].astype(float)

    #features.append('StateHoliday')
    StateHolidayDict= { 0:0 , '0' : 0 , 'a': 1 , 'b' : 2 , 'c' : 3}
    data['StateHoliday'] =  [ StateHolidayDict[i]  for i in data.StateHoliday.values ]

    features.append('DayOfWeek')
    features.append('month')
    features.append('day')
    features.append('year')
    data['Date'] = pd.to_datetime(data.Date)
    data['Date'] = pd.DatetimeIndex(data.Date)
    data = data.join(dates,on='Date')

    #features.append('StoreType')
    StoreTypeDict  = { 'a' : 0 ,'b' : 1 , 'c' : 2 , 'd':3 }
    data['StoreType']  = [ StoreTypeDict[i]  for i in data.StoreType.values ]

    #features.append('Assortment')
    AssortmentDict = { 'a' : 0 ,'b' : 1 , 'c' : 2 }
    data['Assortment'] = [ AssortmentDict[i] for i in data.Assortment.values]
    
    features.append('AssortStoreType')
    data['AssortStoreType'] = data['Assortment'] + 10*data['StoreType']
    
    return data


In [6]:
print("Generate Dates Table")
dates = pd.DataFrame(pd.date_range(train.Date.min(),test.Date.max()),columns=['Date']).set_index('Date')
dates['day']   = dates.index.day.astype(int)
dates['month'] = dates.index.month.astype(int)
dates['year']  = dates.index.year.astype(int)

print("Assume store open, if not provided")
test.fillna(1, inplace=True)

print('Cluster stores by sales correlation.')
sales_pivot  = pd.pivot_table(train,values='Sales',index='Date', columns=['Store'],aggfunc='mean').dropna()
print(sales_pivot.head())
sales_pivot[3].sum()
# 2970073.0
# 4338697.0




Generate Dates Table
Assume store open, if not provided
Cluster stores by sales correlation.
Store         1       2       3        4       5       6       7       8     \
Date                                                                          
2013-01-02  5530.0  4422.0  6823.0   9941.0  4253.0  6089.0  8244.0  5419.0   
2013-01-03  4327.0  4159.0  5902.0   8247.0  3465.0  5398.0  7231.0  4842.0   
2013-01-04  4486.0  4484.0  6069.0   8290.0  4456.0  6092.0  7758.0  4059.0   
2013-01-05  4997.0  2342.0  4523.0  10338.0  1590.0  3872.0  5218.0  2337.0   
2013-01-06     0.0     0.0     0.0      0.0     0.0     0.0     0.0     0.0   

Store         9       10     ...      1106    1107    1108    1109    1110  \
Date                         ...                                             
2013-01-02  4903.0  4812.0   ...    5099.0  3955.0  6220.0  4576.0  4126.0   
2013-01-03  4602.0  4675.0   ...    4330.0  3151.0  4779.0  3654.0  3508.0   
2013-01-04  4798.0  5114.0   ...    3956.

4338697.0

In [7]:
sales_corr   = sales_pivot.corr()
sales_corr

Store,1,2,3,4,5,6,7,8,9,10,...,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115
Store,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.833043,0.882761,0.951319,0.792367,0.859146,0.870347,0.795846,0.892283,0.914628,...,0.851554,0.885964,0.769789,0.890809,0.867332,0.863381,0.905691,0.925063,0.935261,0.914890
2,0.833043,1.000000,0.943403,0.842329,0.954955,0.914589,0.925720,0.938276,0.854018,0.902346,...,0.947622,0.880210,0.950276,0.907091,0.935811,0.926877,0.872769,0.889416,0.823691,0.808041
3,0.882761,0.943403,1.000000,0.866312,0.926369,0.908120,0.934677,0.919314,0.897616,0.914340,...,0.934485,0.925251,0.922113,0.940136,0.932287,0.957384,0.929078,0.897752,0.843933,0.852999
4,0.951319,0.842329,0.866312,1.000000,0.794639,0.840521,0.900652,0.836020,0.916225,0.944953,...,0.872044,0.886220,0.780798,0.872804,0.898905,0.847424,0.870559,0.948669,0.966271,0.928537
5,0.792367,0.954955,0.926369,0.794639,1.000000,0.926716,0.924783,0.928777,0.823604,0.875888,...,0.942647,0.842644,0.966722,0.893254,0.922481,0.918472,0.860604,0.863275,0.778409,0.768393
6,0.859146,0.914589,0.908120,0.840521,0.926716,1.000000,0.906587,0.863868,0.790342,0.884659,...,0.912234,0.834767,0.896992,0.883091,0.883179,0.908468,0.889825,0.886765,0.827918,0.759295
7,0.870347,0.925720,0.934677,0.900652,0.924783,0.906587,1.000000,0.929505,0.897338,0.943562,...,0.959586,0.885001,0.909224,0.910047,0.939107,0.916510,0.874178,0.935976,0.899704,0.859055
8,0.795846,0.938276,0.919314,0.836020,0.928777,0.863868,0.929505,1.000000,0.880306,0.910330,...,0.950998,0.863519,0.948565,0.889421,0.954630,0.898791,0.840348,0.887643,0.819155,0.814436
9,0.892283,0.854018,0.897616,0.916225,0.823604,0.790342,0.897338,0.880306,1.000000,0.936246,...,0.885201,0.905997,0.827621,0.901169,0.914804,0.882752,0.855475,0.902078,0.910479,0.950183


In [8]:
cluster = AgglomerativeClustering(n_clusters=50, linkage='average',affinity=pearson_affinity).fit(sales_corr)
store['SalesCluster'] = cluster.labels_

In [9]:
print('Cluster stores by customer correlation.')
cust_pivot  = pd.pivot_table(train,values='Customers',index='Date', columns=['Store'],aggfunc='mean').dropna()
print(cust_pivot.head())

Cluster stores by customer correlation.
Store        1      2      3       4      5      6      7      8      9     \
Date                                                                         
2013-01-02  668.0  650.0  805.0  1429.0  577.0  781.0  955.0  698.0  481.0   
2013-01-03  578.0  555.0  721.0  1248.0  491.0  689.0  867.0  662.0  453.0   
2013-01-04  619.0  574.0  690.0  1232.0  533.0  724.0  870.0  517.0  497.0   
2013-01-05  635.0  324.0  525.0  1514.0  202.0  467.0  575.0  312.0  450.0   
2013-01-06    0.0    0.0    0.0     0.0    0.0    0.0    0.0    0.0    0.0   

Store        10    ...     1106   1107   1108   1109   1110   1111   1112  \
Date               ...                                                      
2013-01-02  521.0  ...    620.0  525.0  701.0  444.0  507.0  505.0  892.0   
2013-01-03  491.0  ...    550.0  461.0  604.0  392.0  491.0  444.0  809.0   
2013-01-04  517.0  ...    507.0  514.0  640.0  384.0  537.0  453.0  861.0   
2013-01-05  489.0  ...    36

In [10]:
cust_corr   = sales_pivot.corr()
cluster = AgglomerativeClustering(n_clusters=50, linkage='average',affinity=pearson_affinity).fit(cust_corr)
store['CustomerCluster'] = cluster.labels_

In [11]:
# store.head()
print (store[store['SalesCluster']==8])

      Store StoreType Assortment  CompetitionDistance  \
1         2         a          a                570.0   
5         6         a          a                310.0   
7         8         a          a               7520.0   
54       55         a          a                720.0   
59       60         d          c               5540.0   
64       65         a          c              13840.0   
72       73         a          c               1650.0   
86       87         a          a                560.0   
94       95         a          a               1890.0   
120     121         a          a               3570.0   
132     133         a          a                270.0   
166     167         a          a                140.0   
221     222         a          a               2170.0   
235     236         a          a               1000.0   
256     257         a          a                420.0   
265     266         a          c                360.0   
275     276         a          